In [ ]:
# KNOWN ISSUES:
#  * Testing grade levels for prior year assessments (date=7/1/2017) are incorrect
from cyschoolhousesuite import *
import numpy as np
import os
from pathlib import Path
from simple_cysh import *
import xlwings as xw

In [ ]:
sch_ref_path = "Z:\\ChiPrivate\\Chicago Data and Evaluation\\SY18\\SY18 Team Placement\\SY18 FINAL Team Placements.xlsx"
sch_ref_df = pd.read_excel(sch_ref_path, sheet_name="School Data")
xlsx_template = "Z:\\ChiPrivate\\Chicago Reports and Evaluation\\SY18\\Eval Management\\Templates\\CYSH Assessments Template.xlsx"
counts_template_path = 'Z:\\ChiPrivate\\Chicago Data and Evaluation\\SY18\\CYSH Assessment and Enrollment Counts.xlsx'

# Load Assessments, Dates, make 'Assessment_Date' key
assmt_schdl = pd.read_excel('input_files\\CHI-Assessment-Schedule.xlsx')
assmt_schdl.loc[:, 'Date'] = assmt_schdl['Date'].dt.strftime('%m/%d/%Y')
assmt_schdl['Key'] = assmt_schdl['Assessment'] + "_" + assmt_schdl['Date']
assmt_schdl['Friendly Name'] = assmt_schdl['Subject'] + " " + assmt_schdl['Time Period']

In [ ]:
# ## Upload Collins MI-RI
# student_df=get_cysh_df('Student__c', ['Id', 'Name', 'Student_Id__c', 'Grade__c', 'School__c'])
# student_df=student_df.loc[student_df['School__c']=='0011a00000j2XlmAAE']
# assmt_types_df = get_cysh_df(sf_object = 'Picklist_Value__c', sf_fields = ['Id', 'Name', 'Fieldset__c'])

# collins_mi_ri = pd.read_excel(r'Z:\Chicago Collins\School Data\SOY_mi_ri_from_pdfs_.xlsx', sheet_name='Sheet1')
# collins_mi_ri['Name'] = collins_mi_ri['First'] + ' ' + collins_mi_ri['Last']
# collins_mi_ri = collins_mi_ri.merge(student_df, on='Name', how='left')

# # uploaded from ^^ first, then checked second source for entries that weren't uploaded in the first
# collins_mi_ri2 = pd.read_excel(r'Z:\ChiPrivate\Chicago Reports and Evaluation\SY18\SY18 Quarterly Reports\Quarter 2\Quarterly Team Report\Data Source\MI and RI Workbook.xlsx', sheet_name='Sheet1')
# collins_mi_ri2 = collins_mi_ri2.loc[collins_mi_ri2['School']=='Collins Academy High School']

# collins_mi_ri2['School Program'] = collins_mi_ri2['School Program'].str.replace('Tutoring: Literacy', 'ELA')
# collins_mi_ri2['School Program'] = collins_mi_ri2['School Program'].str.replace('Tutoring: Math', 'Math')
# collins_mi_ri2['Key'] = collins_mi_ri2['Student: Student Id'] + collins_mi_ri2['School Program'] + collins_mi_ri2['MI/RI Baseline'].astype(str)

# collins_mi_ri['Key'] = collins_mi_ri['Student_Id__c'] + collins_mi_ri['Type'] + collins_mi_ri['Score'].astype(str)

# collins_mi_ri2 = collins_mi_ri2.loc[~collins_mi_ri2['Key'].isin(collins_mi_ri['Key'])]
# collins_mi_ri2 = collins_mi_ri2.loc[collins_mi_ri2['MI/RI Baseline']!='No Baseline']

# collins_mi_ri2 = collins_mi_ri2.merge(student_df, left_on='Student: Student Id', right_on='Student_Id__c', how='left')

# collins_mi_ri2

# # # a0r1a000001mjFJAAY MI: Math Inventory - MATH
# # # a0r1a000001mjFRAAY RI: Reading Inventory - ELA

# results = []
# for index, row in collins_mi_ri2.iterrows():
#     if row['School Program']=='Math':
#         assmt_type = 'a0r1a000001mjFJAAY'
#     elif row['School Program']=='ELA':
#         assmt_type = 'a0r1a000001mjFRAAY'
#     query_dict = {'Student__c': row['Id'],
#                   'Type__c': assmt_type,
#                   'Date_Administered__c': '2017-09-20',
#                   'Quantile_Score__c': row['MI/RI Baseline'],
#                   'Testing_Grade_Level__c': row['Grade__c']}
#     result = cysh.Assesment__c.create(query_dict)
#     results.append(result)
#     del query_dict, result

In [ ]:
def gather_CYSH_Assessments_from_Schools(sch_ref_df, xlsx_template):
    """
    Input spreadsheet of 'School', 'Informal School Name' (file-friendly), and 'cyconnect folder'.
    Iterates through schools to gather each school's assessment spreadsheet.
    Update file_path construction to suite your cyconnect file structure.
    """
    template_cols = list(pd.read_excel(xlsx_template, index_col=None, skiprows=1))
    all_read_dfs = []
    for index, row in sch_ref_df.iterrows():
        file_path = "Z:\\" + row["cyconnect Folder"] + "\\School Data\\CYSH Assessments - " + row['Informal School Name'] + ".xlsx"
        try:
            read_df = pd.read_excel(file_path, index_col=None, skiprows=1)
        except:
            print("Failed: Loading " + file_path)
            continue
        try:
            read_df = read_df[template_cols]
        except:
            print("Error: Columns don't match template for " + row['Informal School Name'])
            continue
        read_df = read_df[~read_df['School'].isnull()]
        all_read_dfs.append(read_df)
    all_read_dfs = pd.concat(all_read_dfs, axis = 0, ignore_index=True)
    return all_read_dfs

def prep_course_performance_record_df():
    """Get and shape cyschoolhouse report of Course Performance FLs.
    
    Salesforce report is used as official source of Local Student ID and program enrollment. Therefore we can drop duplicates on CY-ID.
    """
    CP_df = get_report("00O1a000002rm2S")
    CP_df.rename(columns={'Student Id':'CY-ID'}, inplace=True)
    
    enrollment_df = CP_df[['CY-ID', 'Program: Program Name']].pivot_table(index='CY-ID', columns='Program: Program Name', aggfunc=len, fill_value=np.nan)
    enrollment_df = enrollment_df.reset_index(level='CY-ID')
    enrollment_df.rename(columns={'Coaching: Attendance': 'ADA Enrollment',
                                  'Tutoring: Literacy': 'ELA Enrollment',
                                  'Tutoring: Math': 'MATH Enrollment'}, inplace=True)
    cols = ['Account Name',
            'Student: Student Name',
            'Student: Grade',
            'Local Student ID',
            'CY-ID']
    CP_df = CP_df[cols]
    CP_df.drop_duplicates(subset='CY-ID', inplace=True)
    CP_df = CP_df.merge(enrollment_df, how='left', on='CY-ID')
    
    return CP_df

def produce_counts(assmt_pivot):
    """Counts all assessments by school."""
    # Count number of assessments for enrolled students
    assmt_counts_ADA = assmt_pivot[assmt_pivot['ADA Enrollment']>0].groupby(by='Account Name')[[x for x in assmt_pivot.columns if 'ADA' in x]].count()
    assmt_counts_ELA = assmt_pivot[assmt_pivot['ELA Enrollment']>0].groupby(by='Account Name')[[x for x in assmt_pivot.columns if 'ELA' in x]].count()
    assmt_counts_MATH = assmt_pivot[assmt_pivot['MATH Enrollment']>0].groupby(by='Account Name')[[x for x in assmt_pivot.columns if 'MATH' in x.upper()]].count()
    assmt_counts = pd.concat([assmt_counts_ADA, assmt_counts_ELA, assmt_counts_MATH], axis=1)

    # sort cols for better baseline col placement
    cols = [x for x in list(assmt_counts) if x not in ['NWEA ELA Baseline', 'NWEA MATH Baseline']]
    cols = cols[0:8] + ['NWEA ELA Baseline'] + cols[8:13] + ['NWEA MATH Baseline'] + cols[13:]
    assmt_counts = assmt_counts[cols]

    # Write counts to Excel
    wb = xw.Book(counts_template_path)
    sht = wb.sheets['Sheet1']
    sht.range('B2:S27').clear_contents()
    sht.range('B1').options(header=True, index=True).value = assmt_counts
    wb.save(counts_template_path)
    wb.close()
    
    return assmt_counts

def load_cysh_assmts(assmt_schdl):
    """Load NWEA and ADA assessments from cyschoolhouse.""" 
    assmt_df = get_report("00O1a000002rlOS")
    assmt_df.rename(columns={'Student_Id':'CY-ID'}, inplace=True)
    assmt_df['Date Administered'] = pd.to_datetime(assmt_df['Date Administered']).dt.strftime('%m/%d/%Y')
    assmt_df['Key'] = assmt_df['Assessment Type: Display Value'] + "_" + assmt_df['Date Administered']
    assmt_df['Student Key'] = assmt_df['CY-ID'] + "_" + assmt_df['Key']
    assmt_df = assmt_df[assmt_df['Key'].isin(assmt_schdl['Key'])]
    assmt_df['Value'] = assmt_df[['Average Daily Attendance', '0 to 300 Scaled Score']].sum(axis=1)
    
    return assmt_df

def pivot_cysh_assmts(CP_df, assmt_df):
    # Pivot assmt_df and merge to CP_df
    assmt_pivot = assmt_df[['CY-ID','Key','Value']].pivot_table(index='CY-ID', columns='Key', values='Value', fill_value=np.nan)
    assmt_pivot = assmt_pivot.reset_index(level='CY-ID')
    assmt_pivot = CP_df.merge(assmt_pivot, how='left', on="CY-ID")
    cols = [x for x in list(assmt_pivot) if x not in list(assmt_schdl['Key'])] + list(assmt_schdl['Key'])
    cols_df = pd.DataFrame(columns=cols)
    assmt_pivot = pd.concat([cols_df,assmt_pivot])[cols]
    assmt_pivot.rename(columns=dict(zip(list(assmt_schdl['Key']), list(assmt_schdl['Friendly Name']))), inplace=True)

    # Add NWEA Baseline cols
    assmt_pivot['NWEA ELA Baseline'] = assmt_pivot[['NWEA ELA Prior Yr', 'NWEA ELA Fall']].bfill(axis=1).iloc[:, 0]
    assmt_pivot['NWEA MATH Baseline'] = assmt_pivot[['NWEA MATH Prior Yr', 'NWEA MATH Fall']].bfill(axis=1).iloc[:, 0]

    return assmt_pivot

In [ ]:
##########################
# Gather New Assessments #
##########################

# Gather cyconnect spreadsheets
SD_df = gather_CYSH_Assessments_from_Schools(sch_ref_df, xlsx_template)
SD_df.to_excel("Z:\\ChiPrivate\\Chicago Reports and Evaluation\\SY18\\SY18 Performance Data Audits\\CYSH Assessments - Whole Site.xlsx", index=False)
SD_df.drop(['Att', 'ELA', 'Math', 'Target', 'Target.1'], axis=1, inplace=True)

# Rename columns
SD_df.columns = list(SD_df)[0:5] + list(assmt_schdl['Key'])
id_cols = ['School', 'Student Name', 'CPS ID', 'CY-ID', 'Grade']

# melt so each row is an assessment score and assessment name
SD_df = SD_df.melt(id_vars=id_cols, var_name='Assessment', value_name='Score')
SD_df = SD_df[~SD_df['Score'].isnull()]
SD_df['Assessment'], SD_df['Assessment Date'] = SD_df['Assessment'].str.split('_', 1).str
SD_df['Key'] = SD_df['CY-ID'] + "_" + SD_df['Assessment'] + "_" + SD_df['Assessment Date']

assmt_df = load_cysh_assmts(assmt_schdl)

# remove CY-ID_assessment-type_assessment-date that occur in assessment report from CYSH
SD_df = SD_df[~SD_df['Key'].isin(assmt_df['Student Key'])]

print("{} new assessments".format(len(SD_df)))

In [ ]:
SD_df.groupby(['School', (SD_df['Assessment'] + "_" + SD_df['Assessment Date'])])[['Score']].count()

In [ ]:
##########################
# Upload New Assessments #
##########################

student_df = get_cysh_df('Student__c', ['Id', 'Student_Id__c'])
assmt_types_df = get_cysh_df('Picklist_Value__c', ['Id', 'Name', 'Fieldset__c'])

new_assmts_df = SD_df.copy()
new_assmts_df['Key'] = new_assmts_df['Assessment'] + "_" + new_assmts_df['Assessment Date']
new_assmts_df = new_assmts_df.merge(student_df, how='left', left_on='CY-ID', right_on='Student_Id__c')
new_assmts_df = new_assmts_df.merge(assmt_types_df, how='left', left_on='Assessment', right_on='Name', suffixes=['_stdnt', '_assmt'])
new_assmts_df = new_assmts_df.merge(assmt_schdl[['Time Period', 'Key']], how='left', on='Key')

new_assmts_df['Assessment Date'] = pd.to_datetime(new_assmts_df['Assessment Date'])
new_assmts_df['Assessment Date'] = new_assmts_df['Assessment Date'].dt.strftime(date_format="%Y-%m-%d")

In [ ]:
results = []
for index, row in new_assmts_df.iterrows():
    query_dict = {'Student__c': row['Id_stdnt'],
                  'Type__c': row['Id_assmt'],
                  'Date_Administered__c': row['Assessment Date']}
    
    if 'ADA Tracker' in row['Assessment']:
        query_dict['Average_Daily_Attendance__c'] = row['Score']
        query_dict['Time_Period__c'] = row['Time Period']
    elif 'NWEA' in row['Assessment']:
        query_dict['X0_to_300_Scaled_Score__c'] = row['Score']
        query_dict['Testing_Grade_Level__c'] = row['Grade']

    result = cysh.Assesment__c.create(query_dict)
    
    results.append(result)
    del query_dict, result

In [ ]:
#########################
# Refresh Spreadshseets #
#########################

# Distribute current CYSH info to School Data folders
CP_df = prep_course_performance_record_df()
# Reload cyschoolhouse assessments, since the previous upload changed the record
assmt_df = load_cysh_assmts(assmt_schdl)
assmt_pivot = pivot_cysh_assmts(CP_df, assmt_df)
assmt_counts = produce_counts(assmt_pivot)

In [ ]:
# Prep for school folders
cols = ['ELA Enrollment', 'MATH Enrollment', 'ADA Enrollment']
assmt_pivot[cols] = assmt_pivot[cols].replace((1, np.nan),('Yes', ''))
assmt_pivot.sort_values(by='Student: Student Name', inplace=True)

# Add NWEA targets
# Known issue: this should not be calculated on baseline, it's a different function if prior yr or fall is the baseline
assmt_pivot['math_key'] = assmt_pivot['Student: Grade'].astype(str) + "_" + assmt_pivot.fillna(0)['NWEA MATH Baseline'].astype(int).astype(str)
assmt_pivot['ela_key'] = assmt_pivot['Student: Grade'].astype(str) + "_" + assmt_pivot.fillna(0)['NWEA ELA Baseline'].astype(int).astype(str)

nwea_targets = pd.read_csv('C:\\Users\\City_Year\\GitHub\\cy-automation-library\\cyautomation\\cyschoolhouse\\input_files\\NWEA_Target_Scores.csv')
nwea_targets['key'] = nwea_targets['Ending Grade'].astype(str) + "_" + nwea_targets['StartRIT'].astype(str)
nwea_targets_math = nwea_targets.loc[nwea_targets['Subject']=='Math']
nwea_targets_math['NWEA MATH Target'] = nwea_targets_math['StartRIT'] + nwea_targets_math['R22']
nwea_targets_ela = nwea_targets.loc[nwea_targets['Subject']=='Reading']
nwea_targets_ela['NWEA ELA Target'] = nwea_targets_ela['StartRIT'] + nwea_targets_ela['R22']

assmt_pivot = assmt_pivot.merge(nwea_targets_math[['NWEA MATH Target', 'key']], left_on='math_key', right_on='key', how='left')
assmt_pivot = assmt_pivot.merge(nwea_targets_ela[['NWEA ELA Target', 'key']], left_on='ela_key', right_on='key', how='left')
assmt_pivot.drop([x for x in list(assmt_pivot)[22:] if 'Target' not in x], axis=1, inplace=True)
assmt_pivot = assmt_pivot[list(assmt_pivot)[0:18] + ['NWEA ELA Target'] + list(assmt_pivot)[18:23]]

# Write Assessment data to school folders
wb = xw.Book(xlsx_template)
sht = wb.sheets['Assessment Data']

for x in assmt_pivot["Account Name"].unique():
    sht.range('A3:Z300').clear_contents()
    sht.range('A3').options(index=False, header=False).value = assmt_pivot[assmt_pivot['Account Name'] == x]
    try:
        wb.save("Z:\\" 
                + sch_ref_df['cyconnect Folder'][sch_ref_df['School'] == x].values[0]
                + "\\School Data\\CYSH Assessments - "
                + sch_ref_df['Informal School Name'][sch_ref_df['School'] == x].values[0]
                + ".xlsx")
    except Exception as e:
        print(f"Failed: Worksheet could not save: {x}: {e}")

wb.close()

In [ ]:
## Delete files in School Data folders
#for x in sch_ref_df['School'].unique():
#    os.remove("Z:\\" 
#              + sch_ref_df['cyconnect Folder'][sch_ref_df['School'] == x].values[0]
#              + "\\School Data\\SY18 cyschoolhouse Assessment Check - "
#              + sch_ref_df['Informal School Name'][sch_ref_df['School'] == x].values[0]
#              + ".xlsx")